![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
import plotly.graph_objs as go

def annualized_sharpe_ratio(trades):
    trade_returns = [
        (trade.exit_price - trade.entry_price) / trade.entry_price 
        for trade in trades
    ]
    mean_trade_return = sum(trade_returns) / len(trade_returns)
    std_trade_return = np.std(trade_returns)
    # 52 because 52 weeks per year and these are weekly trades.
    # Assume 0% risk free rate.
    return mean_trade_return / std_trade_return * np.sqrt(52) 

# Organize the results into a DataFrame.
results = pd.DataFrame(columns=['Annualized Sharpe Ratio'])
optimization = api.read_optimization("O-d29a5dc1cd9f5559a52160c41a2fc1e3")
for optimization_backtest in optimization.backtests.values:
    backtest = api.read_backtest(
        optimization.project_id, optimization_backtest.backtest_id
    )
    stop_loss_percent = backtest.parameter_set.value['stop_loss_percent']
    results.loc[float(stop_loss_percent)] = annualized_sharpe_ratio(
        backtest.total_performance.closed_trades
    )
results.index.name = 'Stop loss percent'

# Show the results.
go.Figure(
    go.Scatter(
        x=results.index * 100, y=results.values.flatten(), 
        name='Annualized Sharpe Ratio'
    ),
    dict(
        title='Stop Loss Performance<br><sub>Results vary significantly so'
            + ' choosing the correct number is a challenge</sub>',
        xaxis=dict(title='Stop Loss Percent (%)'),
        yaxis=dict(title='Annaulized Sharpe Ratio'),
        showlegend=True
    )
).show()
results